In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7576,2021-02-18T17:00:00,ITA,19,Sicilia,38.115697,13.362357,930,145,1075,31929,...,NaN,6.0,NaN,NaN,147665.0,0.0,1665126.0,540628.0,ITG,ITG1
7577,2021-02-18T17:00:00,ITA,9,Toscana,43.769231,11.255889,717,144,861,11942,...,NaN,20.0,NaN,NaN,145589.0,387.0,2331158.0,193631.0,ITI,ITI1
7578,2021-02-18T17:00:00,ITA,10,Umbria,43.106758,12.388247,469,85,554,7864,...,"Si fa presente che 20 dei ricoveri NON UTI, so...",10.0,NaN,NaN,42058.0,0.0,661246.0,95157.0,ITI,ITI2
7579,2021-02-18T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,10,2,12,124,...,NaN,0.0,NaN,NaN,7947.0,0.0,72352.0,2479.0,ITC,ITC2


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note', 'note_test', 'note_casi', 'codice_nuts_1', 'codice_nuts_2']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7576,2021-02-18T17:00:00,19,Sicilia,930,145,1075,31929,33004,-651,480,...,NaN,NaN,147665,2205754,1080729.0,6.0,147665.0,0.0,1665126.0,540628.0
7577,2021-02-18T17:00:00,9,Toscana,717,144,861,11942,12803,407,956,...,NaN,NaN,145976,2524789,1337126.0,20.0,145589.0,387.0,2331158.0,193631.0
7578,2021-02-18T17:00:00,10,Umbria,469,85,554,7864,8418,-13,319,...,NaN,NaN,42058,756403,305182.0,10.0,42058.0,0.0,661246.0,95157.0
7579,2021-02-18T17:00:00,2,Valle d'Aosta,10,2,12,124,136,5,11,...,NaN,NaN,7947,74831,42980.0,0.0,7947.0,0.0,72352.0,2479.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-02-18', '2021-02-17')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-18T17:00:00,1,Piemonte,1866,128,1994,10012,12006,-161,501,...,NaN,NaN,236075,2375443,1187837.0,4.0,232279.0,3796.0,1997256.0,378187.0
1,2021-02-18T17:00:00,2,Valle d'Aosta,10,2,12,124,136,5,11,...,NaN,NaN,7947,74831,42980.0,0.0,7947.0,0.0,72352.0,2479.0
2,2021-02-18T17:00:00,3,Lombardia,3716,368,4084,45889,49973,1280,2540,...,NaN,NaN,570023,6235973,2992098.0,29.0,565413.0,4610.0,5943704.0,292269.0
3,2021-02-18T17:00:00,5,Veneto,830,98,928,21058,21986,-1392,1042,...,NaN,NaN,323888,4689379,1368822.0,7.0,321135.0,2753.0,3929969.0,759410.0
4,2021-02-18T17:00:00,6,Friuli Venezia Giulia,361,58,419,8991,9410,-2,262,...,NaN,NaN,73043,1273548,457337.0,6.0,69019.0,4024.0,1202324.0,71224.0
5,2021-02-18T17:00:00,7,Liguria,541,59,600,4210,4810,-46,334,...,NaN,NaN,74873,969946,415469.0,4.0,74873.0,0.0,888441.0,81505.0
6,2021-02-18T17:00:00,8,Emilia-Romagna,1890,181,2071,32858,34929,-339,1565,...,NaN,NaN,240650,3541280,1443179.0,18.0,240569.0,81.0,3237722.0,303558.0
7,2021-02-18T17:00:00,9,Toscana,717,144,861,11942,12803,407,956,...,NaN,NaN,145976,2524789,1337126.0,20.0,145589.0,387.0,2331158.0,193631.0
8,2021-02-18T17:00:00,10,Umbria,469,85,554,7864,8418,-13,319,...,NaN,NaN,42058,756403,305182.0,10.0,42058.0,0.0,661246.0,95157.0
9,2021-02-18T17:00:00,11,Marche,520,79,599,7757,8356,201,532,...,NaN,NaN,62216,776442,469765.0,4.0,62216.0,0.0,727639.0,48803.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-17T17:00:00,1,Piemonte,1875,138,2013,10154,12167,-59,959,...,NaN,NaN,235574,2361005,1184276.0,4.0,231893.0,3681.0,1991465.0,369540.0
1,2021-02-17T17:00:00,2,Valle d'Aosta,8,2,10,121,131,5,14,...,NaN,NaN,7936,74548,42882.0,0.0,7936.0,0.0,72101.0,2447.0
2,2021-02-17T17:00:00,3,Lombardia,3740,363,4103,44590,48693,20,1764,...,NaN,NaN,567483,6193465,2980514.0,8.0,563153.0,4330.0,5913564.0,279901.0
3,2021-02-17T17:00:00,5,Veneto,875,97,972,22406,23378,-875,492,...,NaN,NaN,322846,4656125,1365422.0,10.0,320311.0,2535.0,3918407.0,737718.0
4,2021-02-17T17:00:00,6,Friuli Venezia Giulia,384,59,443,8969,9412,-59,333,...,NaN,NaN,72781,1265663,454819.0,2.0,68822.0,3959.0,1196858.0,68805.0
5,2021-02-17T17:00:00,7,Liguria,569,60,629,4227,4856,-104,309,...,NaN,NaN,74539,962220,412417.0,3.0,74539.0,0.0,883920.0,78300.0
6,2021-02-17T17:00:00,8,Emilia-Romagna,1902,176,2078,33190,35268,-1769,1025,...,NaN,NaN,239091,3511647,1437392.0,12.0,239015.0,76.0,3221988.0,289659.0
7,2021-02-17T17:00:00,9,Toscana,699,137,836,11560,12396,85,773,...,NaN,NaN,145020,2503762,1325740.0,13.0,144671.0,349.0,2318256.0,185506.0
8,2021-02-17T17:00:00,10,Umbria,471,83,554,7877,8431,146,438,...,NaN,NaN,41739,749120,304193.0,5.0,41739.0,0.0,657404.0,91716.0
9,2021-02-17T17:00:00,11,Marche,534,78,612,7543,8155,150,471,...,NaN,NaN,61684,769043,464771.0,5.0,61684.0,0.0,722905.0,46138.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-18T17:00:00,1,Piemonte,1866,128,1994,10012,12006,-161,501,...,NaN,NaN,236075,2375443,1187837.0,4.0,232279.0,3796.0,1997256.0,378187.0
1,2021-02-18T17:00:00,2,Valle d'Aosta,10,2,12,124,136,5,11,...,NaN,NaN,7947,74831,42980.0,0.0,7947.0,0.0,72352.0,2479.0
2,2021-02-18T17:00:00,3,Lombardia,3716,368,4084,45889,49973,1280,2540,...,NaN,NaN,570023,6235973,2992098.0,29.0,565413.0,4610.0,5943704.0,292269.0
3,2021-02-18T17:00:00,4,Trentino-Alto Adige,425,64,489,10263,10752,153,770,...,0.0,0.0,81431,1223282,359085.0,6.0,67194.0,14237.0,1008215.0,215067.0
4,2021-02-18T17:00:00,5,Veneto,830,98,928,21058,21986,-1392,1042,...,NaN,NaN,323888,4689379,1368822.0,7.0,321135.0,2753.0,3929969.0,759410.0
5,2021-02-18T17:00:00,6,Friuli Venezia Giulia,361,58,419,8991,9410,-2,262,...,NaN,NaN,73043,1273548,457337.0,6.0,69019.0,4024.0,1202324.0,71224.0
6,2021-02-18T17:00:00,7,Liguria,541,59,600,4210,4810,-46,334,...,NaN,NaN,74873,969946,415469.0,4.0,74873.0,0.0,888441.0,81505.0
7,2021-02-18T17:00:00,8,Emilia-Romagna,1890,181,2071,32858,34929,-339,1565,...,NaN,NaN,240650,3541280,1443179.0,18.0,240569.0,81.0,3237722.0,303558.0
8,2021-02-18T17:00:00,9,Toscana,717,144,861,11942,12803,407,956,...,NaN,NaN,145976,2524789,1337126.0,20.0,145589.0,387.0,2331158.0,193631.0
9,2021-02-18T17:00:00,10,Umbria,469,85,554,7864,8418,-13,319,...,NaN,NaN,42058,756403,305182.0,10.0,42058.0,0.0,661246.0,95157.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,0,2021-02-17T17:00:00,1,Piemonte,1875,138,2013,10154,12167,-59,...,NaN,NaN,235574,2361005,1184276.0,4.0,231893.0,3681.0,1991465.0,369540.0
1,1,2021-02-17T17:00:00,2,Valle d'Aosta,8,2,10,121,131,5,...,NaN,NaN,7936,74548,42882.0,0.0,7936.0,0.0,72101.0,2447.0
2,2,2021-02-17T17:00:00,3,Lombardia,3740,363,4103,44590,48693,20,...,NaN,NaN,567483,6193465,2980514.0,8.0,563153.0,4330.0,5913564.0,279901.0
3,19,2021-02-17T17:00:00,4,Trentino-Alto Adige,416,65,481,10118,10599,-67,...,0.0,0.0,80661,1207930,358111.0,3.0,66868.0,13793.0,1003939.0,203991.0
4,3,2021-02-17T17:00:00,5,Veneto,875,97,972,22406,23378,-875,...,NaN,NaN,322846,4656125,1365422.0,10.0,320311.0,2535.0,3918407.0,737718.0
5,4,2021-02-17T17:00:00,6,Friuli Venezia Giulia,384,59,443,8969,9412,-59,...,NaN,NaN,72781,1265663,454819.0,2.0,68822.0,3959.0,1196858.0,68805.0
6,5,2021-02-17T17:00:00,7,Liguria,569,60,629,4227,4856,-104,...,NaN,NaN,74539,962220,412417.0,3.0,74539.0,0.0,883920.0,78300.0
7,6,2021-02-17T17:00:00,8,Emilia-Romagna,1902,176,2078,33190,35268,-1769,...,NaN,NaN,239091,3511647,1437392.0,12.0,239015.0,76.0,3221988.0,289659.0
8,7,2021-02-17T17:00:00,9,Toscana,699,137,836,11560,12396,85,...,NaN,NaN,145020,2503762,1325740.0,13.0,144671.0,349.0,2318256.0,185506.0
9,8,2021-02-17T17:00:00,10,Umbria,471,83,554,7877,8431,146,...,NaN,NaN,41739,749120,304193.0,5.0,41739.0,0.0,657404.0,91716.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione
0,2021-02-18T17:00:00,1,Piemonte,1866,128,1994,10012,12006,-161,501,...,NaN,236075,2375443,1187837.0,4.0,232279.0,3796.0,1997256.0,378187.0,4341375
1,2021-02-18T17:00:00,2,Valle d'Aosta,10,2,12,124,136,5,11,...,NaN,7947,74831,42980.0,0.0,7947.0,0.0,72352.0,2479.0,125501
2,2021-02-18T17:00:00,3,Lombardia,3716,368,4084,45889,49973,1280,2540,...,NaN,570023,6235973,2992098.0,29.0,565413.0,4610.0,5943704.0,292269.0,10103969
3,2021-02-18T17:00:00,4,Trentino-Alto Adige,425,64,489,10263,10752,153,770,...,0.0,81431,1223282,359085.0,6.0,67194.0,14237.0,1008215.0,215067.0,1074819
4,2021-02-18T17:00:00,5,Veneto,830,98,928,21058,21986,-1392,1042,...,NaN,323888,4689379,1368822.0,7.0,321135.0,2753.0,3929969.0,759410.0,4907704
5,2021-02-18T17:00:00,6,Friuli Venezia Giulia,361,58,419,8991,9410,-2,262,...,NaN,73043,1273548,457337.0,6.0,69019.0,4024.0,1202324.0,71224.0,1211357
6,2021-02-18T17:00:00,7,Liguria,541,59,600,4210,4810,-46,334,...,NaN,74873,969946,415469.0,4.0,74873.0,0.0,888441.0,81505.0,1543127
7,2021-02-18T17:00:00,8,Emilia-Romagna,1890,181,2071,32858,34929,-339,1565,...,NaN,240650,3541280,1443179.0,18.0,240569.0,81.0,3237722.0,303558.0,4467118
8,2021-02-18T17:00:00,9,Toscana,717,144,861,11942,12803,407,956,...,NaN,145976,2524789,1337126.0,20.0,145589.0,387.0,2331158.0,193631.0,3722729
9,2021-02-18T17:00:00,10,Umbria,469,85,554,7864,8418,-13,319,...,NaN,42058,756403,305182.0,10.0,42058.0,0.0,661246.0,95157.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione,totale_posti_terapia_intensiva
0,2021-02-18T17:00:00,1,Piemonte,1866,128,1994,10012,12006,-161,501,...,236075,2375443,1187837.0,4.0,232279.0,3796.0,1997256.0,378187.0,4341375,575
1,2021-02-18T17:00:00,2,Valle d'Aosta,10,2,12,124,136,5,11,...,7947,74831,42980.0,0.0,7947.0,0.0,72352.0,2479.0,125501,20
2,2021-02-18T17:00:00,3,Lombardia,3716,368,4084,45889,49973,1280,2540,...,570023,6235973,2992098.0,29.0,565413.0,4610.0,5943704.0,292269.0,10103969,1036
3,2021-02-18T17:00:00,4,Trentino-Alto Adige,425,64,489,10263,10752,153,770,...,81431,1223282,359085.0,6.0,67194.0,14237.0,1008215.0,215067.0,1074819,106
4,2021-02-18T17:00:00,5,Veneto,830,98,928,21058,21986,-1392,1042,...,323888,4689379,1368822.0,7.0,321135.0,2753.0,3929969.0,759410.0,4907704,1016
5,2021-02-18T17:00:00,6,Friuli Venezia Giulia,361,58,419,8991,9410,-2,262,...,73043,1273548,457337.0,6.0,69019.0,4024.0,1202324.0,71224.0,1211357,180
6,2021-02-18T17:00:00,7,Liguria,541,59,600,4210,4810,-46,334,...,74873,969946,415469.0,4.0,74873.0,0.0,888441.0,81505.0,1543127,209
7,2021-02-18T17:00:00,8,Emilia-Romagna,1890,181,2071,32858,34929,-339,1565,...,240650,3541280,1443179.0,18.0,240569.0,81.0,3237722.0,303558.0,4467118,634
8,2021-02-18T17:00:00,9,Toscana,717,144,861,11942,12803,407,956,...,145976,2524789,1337126.0,20.0,145589.0,387.0,2331158.0,193631.0,3722729,523
9,2021-02-18T17:00:00,10,Umbria,469,85,554,7864,8418,-13,319,...,42058,756403,305182.0,10.0,42058.0,0.0,661246.0,95157.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-02-18T17:00:00,3,Lombardia,3716,368,4084,45889,49973,1280,2540,...,35.52,1299,776,42508,11584.0,55,1205,5.98,0.02514,5.64
1,2021-02-18T17:00:00,15,Campania,1290,110,1400,67782,69182,-497,1573,...,21.78,-489,-2,19886,13007.0,29,2041,7.91,0.02719,4.29
2,2021-02-18T17:00:00,8,Emilia-Romagna,1890,181,2071,32858,34929,-339,1565,...,28.55,-332,540,29633,5787.0,27,1871,5.28,0.03503,5.39
3,2021-02-18T17:00:00,5,Veneto,830,98,928,21058,21986,-1392,1042,...,9.65,-1348,550,33254,3400.0,30,2404,3.13,0.02123,6.60
4,2021-02-18T17:00:00,12,Lazio,1945,232,2177,33461,35638,-969,1025,...,27.39,-863,154,29941,17100.0,41,1953,3.42,0.01747,3.80
5,2021-02-18T17:00:00,9,Toscana,717,144,861,11942,12803,407,956,...,27.53,382,183,21027,11386.0,11,538,4.55,0.02568,3.92
6,2021-02-18T17:00:00,16,Puglia,1283,159,1442,34287,35729,-2305,844,...,43.09,-2277,-39,10033,4152.0,37,3112,8.41,0.02106,3.44
7,2021-02-18T17:00:00,4,Trentino-Alto Adige,425,64,489,10263,10752,153,770,...,60.38,145,9,15352,974.0,7,610,5.02,0.07164,7.58
8,2021-02-18T17:00:00,11,Marche,520,79,599,7757,8356,201,532,...,55.24,214,61,7399,4994.0,17,314,7.19,0.03504,4.10
9,2021-02-18T17:00:00,1,Piemonte,1866,128,1994,10012,12006,-161,501,...,22.26,-142,-458,14438,3561.0,16,646,3.47,0.01154,5.44


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Umbria,85,32,117,72.65
1,Trentino-Alto Adige,64,42,106,60.38
2,Marche,79,64,143,55.24
3,Abruzzo,70,78,148,47.30
4,Puglia,159,210,369,43.09
5,Lombardia,368,668,1036,35.52
6,Molise,12,22,34,35.29
7,Friuli Venezia Giulia,58,122,180,32.22
8,Emilia-Romagna,181,453,634,28.55
9,Liguria,59,150,209,28.23


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,2021-02-16T17:00:00,22,P.A. Trento,168,27,195,2522,2717,34,195,...,NaN,NaN,30552,571258,166147.0,3.0,26192.0,4360.0,534809.0,36449.0
718,2021-02-17T17:00:00,21,P.A. Bolzano,254,39,293,7519,7812,-137,450,...,NaN,NaN,49798,632885,191540.0,0.0,40614.0,9184.0,466970.0,165915.0
719,2021-02-17T17:00:00,22,P.A. Trento,162,26,188,2599,2787,70,311,...,NaN,NaN,30863,575045,166571.0,3.0,26254.0,4609.0,536969.0,38076.0
720,2021-02-18T17:00:00,21,P.A. Bolzano,253,37,290,7563,7853,41,478,...,NaN,NaN,50276,644717,192208.0,3.0,40879.0,9397.0,469657.0,175060.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,2021-02-14T17:00:00,4,Trentino-Alto Adige,419,66,485,10662,11147,23,562,...,9.0,8312.0,882.0,1.0,13.61,4.35,6.76,63.72,4.76,0.21
357,2021-02-15T17:00:00,4,Trentino-Alto Adige,432,66,498,10516,11014,-133,238,...,5.0,2813.0,561.0,-146.0,13.25,4.52,8.46,42.42,0.00,-1.19
358,2021-02-16T17:00:00,4,Trentino-Alto Adige,422,66,488,10178,10666,-348,759,...,9.0,14430.0,644.0,-338.0,13.52,4.58,5.26,117.86,0.00,-3.16
359,2021-02-17T17:00:00,4,Trentino-Alto Adige,416,65,481,10118,10599,-67,761,...,9.0,13170.0,1071.0,-60.0,13.51,4.54,5.78,71.06,-1.52,-0.63


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
